* 동일한 완료를 여러 번 요청하는 경우 LLM 공급자에 대한 API 호출 횟수를 줄여 비용을 절감 할 수 있습니다.
* LLM 제공업체에 대한 API 호출 횟수를 줄여 애플리케이션의 속도를 높일 수 있습니다.

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

In [3]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("{actor} 에 대해서 200자 내외로 요약해줘")
prompt

PromptTemplate(input_variables=['actor'], template='{actor} 에 대해서 200자 내외로 요약해줘')

In [4]:
chain = prompt | llm

In [5]:
%time response = chain.invoke({"actor": "톰 홀랜드"})
print(response.content)

CPU times: user 20.5 ms, sys: 9.73 ms, total: 30.3 ms
Wall time: 3.66 s
톰 홀랜드(Tom Holland)는 영국 출신의 배우로, 1996년 6월 1일에 태어났다. 그는 2016년 영화 "캡틴 아메리카: 시빌 워"에서 스파이더맨 역할로 큰 주목을 받았고, 이후 마블 시네마틱 유니버스(MCU)의 여러 작품에서 같은 역할을 이어갔다. 무용과 연기 경력이 돋보이며, "빌리 엘리어트 뮤지컬"에서 주연을 맡기도 했다. 그의 연기력과 다재다능함으로 인해 현재 할리우드에서 가장 촉망받는 젊은 배우 중 한 명으로 꼽힌다.


In [6]:
from langchain.cache import SQLiteCache
from langchain.globals import set_llm_cache

set_llm_cache(SQLiteCache(database_path="my_llm_cache.db"))

In [11]:
%time response = chain.invoke({"actor": "젠데이아"})
print(response.content)

CPU times: user 2.32 ms, sys: 731 µs, total: 3.05 ms
Wall time: 2.43 ms
젠데이아(Zendaya)는 미국의 배우, 가수, 모델로, 1996년 9월 1일 캘리포니아주 오클랜드에서 태어났습니다. 디즈니 채널 시리즈 "Shake It Up"으로 주목받기 시작했으며, 이후 HBO 드라마 "유포리아(Euphoria)"에서의 뛰어난 연기로 에미상을 수상했습니다. 또한, "스파이더맨" 시리즈 등 다양한 영화에서도 활약하며, 헐리우드에서 다재다능한 인물로 자리매김하고 있습니다. 젠데이아는 패션 아이콘으로도 알려져 있으며, 다양한 사회적 이슈에 대해 목소리를 내는 활동가이기도 합니다.


## 토큰 사용량 확인
* 요금 등등

In [16]:
from langchain.callbacks import get_openai_callback

llm = ChatOpenAI(model="gpt-4o", temperature=0)

with get_openai_callback() as cb:
    result = llm.invoke("스파이더맨2를 요약해줘")
    print(result.content)
    print(cb)

"스파이더맨 2"는 2004년에 개봉한 슈퍼히어로 영화로, 샘 레이미가 감독하고 토비 맥과이어가 주연을 맡았습니다. 이 영화는 2002년에 개봉한 "스파이더맨"의 후속작으로, 마블 코믹스의 캐릭터 스파이더맨을 기반으로 하고 있습니다.

영화는 피터 파커(스파이더맨)가 자신의 슈퍼히어로 정체성과 일상 생활 사이에서 균형을 맞추기 위해 고군분투하는 이야기를 중심으로 전개됩니다. 피터는 메리 제인 왓슨과의 관계, 학업, 그리고 일상적인 문제들로 인해 스트레스를 받으며, 이로 인해 그의 능력이 일시적으로 약해지기도 합니다.

한편, 과학자 오토 옥타비우스는 자신의 실험이 실패하면서 강력한 기계 팔을 가진 닥터 옥토퍼스로 변하게 됩니다. 닥터 옥토퍼스는 뉴욕 시를 위협하며, 피터는 다시 한 번 스파이더맨으로서의 책임을 다하기 위해 나서야 합니다.

영화는 피터가 자신의 정체성과 책임을 받아들이고, 닥터 옥토퍼스를 물리치며, 메리 제인과의 관계를 회복하는 과정을 그립니다. "스파이더맨 2"는 뛰어난 액션 장면과 깊이 있는 캐릭터 개발로 많은 찬사를 받았으며, 슈퍼히어로 영화의 대표작 중 하나로 꼽힙니다.
Tokens Used: 383
	Prompt Tokens: 18
	Completion Tokens: 365
Successful Requests: 1
Total Cost (USD): $0.005565


## 모델 직렬화(Serialization) - 저장 및 불러오기

In [17]:
from langchain.llms.loading import load_llm

print(f"ChatOpenAI: {ChatOpenAI.is_lc_serializable()}")

ChatOpenAI: True


In [18]:
llm = ChatOpenAI(model="gpt-4o", temperature=0)
prompt = PromptTemplate.from_template("{movie} 요약해줘")
prompt

PromptTemplate(input_variables=['movie'], template='{movie} 요약해줘')

In [19]:
chain = prompt | llm
chain.is_lc_serializable()

True

In [20]:
from langchain.load import dumpd

dumped_chain = dumpd(chain)
dumped_chain

{'lc': 1,
 'type': 'constructor',
 'id': ['langchain', 'schema', 'runnable', 'RunnableSequence'],
 'kwargs': {'first': {'lc': 1,
   'type': 'constructor',
   'id': ['langchain', 'prompts', 'prompt', 'PromptTemplate'],
   'kwargs': {'input_variables': ['movie'],
    'template': '{movie} 요약해줘',
    'template_format': 'f-string'},
   'name': 'PromptTemplate',
   'graph': {'nodes': [{'id': 0, 'type': 'schema', 'data': 'PromptInput'},
     {'id': 1,
      'type': 'runnable',
      'data': {'id': ['langchain', 'prompts', 'prompt', 'PromptTemplate'],
       'name': 'PromptTemplate'}},
     {'id': 2, 'type': 'schema', 'data': 'PromptTemplateOutput'}],
    'edges': [{'source': 0, 'target': 1}, {'source': 1, 'target': 2}]}},
  'last': {'lc': 1,
   'type': 'constructor',
   'id': ['langchain', 'chat_models', 'openai', 'ChatOpenAI'],
   'kwargs': {'model_name': 'gpt-4o',
    'temperature': 0.0,
    'openai_api_key': {'lc': 1, 'type': 'secret', 'id': ['OPENAI_API_KEY']},
    'openai_proxy': '',
   

In [21]:
from langchain.load.load import load

loaded_chain = load(dumped_chain)
print(loaded_chain.invoke({"movie": "스파이더맨1"}))

content='"스파이더맨 1"은 2002년에 개봉한 슈퍼히어로 영화로, 샘 레이미가 감독하고 토비 맥과이어가 주연을 맡았습니다. 이 영화는 마블 코믹스의 인기 캐릭터 스파이더맨의 기원을 다루고 있습니다.\n\n이야기는 뉴욕에 사는 평범한 고등학생 피터 파커(토비 맥과이어)가 유전자 변형된 거미에게 물리면서 시작됩니다. 이 사건으로 인해 피터는 초인적인 힘, 벽을 타고 오를 수 있는 능력, 그리고 거미줄을 발사할 수 있는 능력을 얻게 됩니다. 처음에는 자신의 능력을 어떻게 사용할지 몰라 혼란스러워하지만, 삼촌 벤(클리프 로버트슨)의 죽음 이후 "큰 힘에는 큰 책임이 따른다"는 교훈을 마음에 새기고 스파이더맨으로서 범죄와 싸우기로 결심합니다.\n\n한편, 피터의 친구 해리 오스본(제임스 프랭코)의 아버지이자 대기업 오스코프의 CEO인 노먼 오스본(윌렘 대포)은 자신이 개발한 실험용 약물을 사용하다가 그린 고블린이라는 악당으로 변하게 됩니다. 그린 고블린은 뉴욕을 위협하며 스파이더맨과 여러 차례 충돌하게 됩니다.\n\n영화는 피터가 스파이더맨으로서의 정체성을 확립하고, 그린 고블린과의 최종 대결을 통해 자신의 능력과 책임을 받아들이는 과정을 그립니다. 마지막에는 피터가 사랑하는 사람들을 보호하기 위해 자신의 정체를 숨기기로 결심하며, 메리 제인 왓슨(커스틴 던스트)과의 관계도 복잡해집니다.\n\n"스파이더맨 1"은 개봉 당시 큰 인기를 끌었으며, 이후 여러 후속작과 리부트 시리즈로 이어졌습니다.' response_metadata={'token_usage': {'completion_tokens': 454, 'prompt_tokens': 17, 'total_tokens': 471}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_729ea513f7', 'finish_reason': 'stop', 'logprobs': None} id='run-e57b4c4a-70bb-4693-b33b-48d37f920cdc-0'
